In [1]:
import pandas as pd
from time import sleep
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import chromedriver_binary
import requests
import pprint

In [2]:
def get_login():
    '''
    Access API keys from external file and placed in a list.

    Parameters
    ----------
    None:

    Returns
    ----------
    API_KEYS: (list)
        Return API keys used for connected to Spotify and Genius APIs.
    '''
    f = open('../data/LI_login.txt', 'r')
    creds = f.readlines()

    for idx, key in enumerate(creds):
        creds[idx] = key.replace('\n', '')

    return creds

In [3]:
def stock_scraper(stock):
    '''
    Scrapes passed in url (based on Yahoo Finance). Uses Selenium to
    complete loading stock history table.
    
    Parameters
    ----------
    stock : (str)
        A url to a webpage with a lazy loaded stock history table.

    Returns
    ----------
    mongo_dict : (dict)
        Dictionary ready for Mongo insert. Keys are dates, values are 
        lists of each days EOD trading results. 
    '''
    driver = webdriver.Chrome()
    driver.get(stock)
    sleep(10)
    for _ in range(30):
        sleep(1.5)          #random.randint for more randomized scrapping intervals
        driver.execute_script("window.scrollTo(0, window.scrollY + 3000)")
    r = driver.page_source
    soup = BeautifulSoup(r, 'html.parser')
    table_soup = soup.find_all('table')
    indices = table_soup[0].find_all('td')
    indices.pop(-1)
    mongo_dict = mongo_parser(indices)
    
    if mongo_dict: 
        driver.close()
        
    return mongo_dict

In [4]:
def li_login():
    email, pw = get_login()
    driver = webdriver.Chrome()
    driver.get('https://www.linkedin.com/')
    driver.find_element_by_id('session_key').send_keys(email)
    driver.find_element_by_id('session_password').send_keys(pw)
    driver.find_element_by_id('Sign in').click()

In [7]:
!ls ../data/

LI_login.txt


In [5]:
li_login()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="Sign in"]"}
  (Session info: chrome=84.0.4147.89)
